---
## 3.5.0 背景
---

上一节我们讲到了**边缘似然函数**。这一节，我们通过最大化边缘似然函数，来确定超参数的值。这样的方法在统计学中叫做经验贝叶斯、第二类最大似然、或者叫做推广的最大似然。

- 一个参数$w$：$1\times M$维的向量
- 两个超参数：
  +  $\alpha:$精度，零均值同分布$p(w|\alpha)=N(w|0, \alpha^{-1}I)$
  +  $\beta:$噪声，$p(t|x,w,\beta)=N(t|y(x, w), \beta^-1)$
  
我们引入超参数$\alpha, \beta$的先验分布，那么预测分布可以通过对$w,\alpha, \beta$求积分得到：

$$
p(t|\mathrm{\bf{t}})=\iiint p(t,w,\alpha,\beta|t)\mathrm{dwd\alpha d\beta}=\iiint p(t|w,\beta)p(w|t,\alpha, \beta)p(\alpha, \beta|t)\mathrm{dwd\alpha d\beta}
$$

<font color='blue'>
- 其中积分的第一项$p(t|w,\beta)=p(t|x,w,\beta)=N(t|y(x, w), \beta^-1)$，这里为了方便，我们忽略了$x$
- 积分的第二项$p(w|t,\alpha, \beta)=p(w|t)=N(w|m_N,S_N)$
- 如果后验分布$p(\alpha, \beta|t)$在$\hat{\alpha},\hat{\beta}$有尖锋，那么$p(t|\mathrm{\bf{t}})\simeq p(t|\mathrm{\bf{t}},\hat{\alpha},\hat{\beta})=\int p(t|w,\hat{\beta})p(w|t,\hat{\alpha}, \hat{\beta})\mathrm{dw}$
- 根据贝叶斯定理 $p(\alpha, \beta|\mathrm{\bf{t}}) \propto p(\mathrm{\bf{t}}|\alpha, \beta)p(\alpha, \beta)$

---
## 3.5.1 计算证据函数
---
边缘似然函数是通过对权值参数$w$进⾏积分得到的，即
$$
p(\mathrm{\bf{t}}|\alpha,\beta)=\int p(\mathrm{\bf{t}}|w,\alpha,\beta)p(w|\alpha, \beta)\mathrm{\bf{w}} \int p(\mathrm{\bf{t}}|w,\beta)p(w|\alpha)\mathrm{\bf{w}}
$$

- $p(\mathrm{\bf{t}}|w,\beta)=\prod_{i=1}^N N(\mathrm{\bf{t}}_n|w^T\phi(x_n), \beta^{-1})$
- $p(w|\alpha)=N(w|0, \alpha^{-1}I)$

所以有：

$$
p(\mathrm{\bf{t}}|\alpha,\beta)=\left(\frac{\beta}{2\pi}\right)^{N/2}\left(\frac{\alpha}{2\pi}\right)^{M/2}\int \mathrm{exp}(-E(w))\mathrm{dw}\\
E(w)=\beta E_D(w)+\alpha E_W(w)=\frac{\beta}{2}||t-\Phi w||^2+\frac{\alpha}{2}w^Tw
$$

那么经过一系列计算之后，可以得到：
$$
\ln p(\textbf{t}|\alpha,\beta) = \frac{M}{2}\ln \alpha + \frac{N}{2}\ln \beta - E(m_N) - \frac{1}{2}\ln\vert A \vert - \frac{N}{2}\ln(2\pi) \tag{3.86}
$$

<font color='blue'>
- $M$是参数$w$的维度，$N$是样本的数目
- $A=\alpha I + \beta \Phi^T \Phi$

回到多项式回归问题，我们可以画出模型证据与多项式阶数之间的关系，如图3.14所示。

![](https://raw.githubusercontent.com/data2world/PRML_Note/master/IMG/CH03/3.14.png)


---
## 3.5.2 最大化证据函数
---


定义一个特征向量方程：
$$
(\beta\Phi^T\Phi)u_i = \lambda_iu_i \tag{3.87}
$$

对方阵$M \times M \text{的} A$。

对$\alpha$，最大化$p(t|\alpha, \beta)$

- $\alpha = \frac{\lambda}{m_N^Tm_N}$
- $\gamma=\sum_i\frac{\lambda_i}{\alpha+\lambda_i}$
- 矩阵$A$是$M \times M$的方阵， $A=\alpha I + \beta\Phi^T\Phi$
- $\alpha = \frac{\lambda}{m^T_Nm_N}$ 这是一个隐式解，求解方法使用迭代。

<font color = 'blue'>
>**初始化选择$\alpha$，然后使用公式$m_N=\beta A^{-1} \Phi^{T}t$计算$m_N$, 然后计算$A=\alpha I + \beta \Phi^{T} \Phi$，此处的$A=S^{-1}$**
</font>


对噪声参数$\beta$可以使用同样的方法。

---
## 3.5.3 参数的有效数量
---

![](https://raw.githubusercontent.com/data2world/PRML_Note/master/IMG/CH03/3.15.png)


如上图所示：
- 绿色的线表示先验，红色的表示边际似然函数。
- 同样的有$\alpha = \frac{\lambda}{m_N^Tm_N}$，$\gamma=\sum_i\frac{\lambda_i}{\alpha+\lambda_i}$，$\lambda_i$是Hessian矩阵$\Phi^T\Phi$的特征值
- 特征值$\lambda_i$度量了似然函数的曲率
- 当参数$\gamma=1$时，对应的参数$w_i$与最大似然接近，那么这样的参数有着良好的性质。